## Название документа

Название НПА – это краткий текст, описывающий основную суть и тему документа.
Название часто начинается с предлогов «о» или «об». В тексте НПА название может встречаться
в кавычках или без кавычек. Название не включает в себя тип НПА.

Для примера название документа – это «О заместителе Министра внутренних дел
Российской Федерации – начальнике Следственного департамента Министерства внутренних дел Российской Федерации».

Поле name должно содержать строку с названием в кавычках или без, в зависимости от того, как оно встретилось в тексте НПА. При сравнении предсказанного и настоящего номера в проверяющей системе не учитывается регистр букв, пробелы и символы перевода строки не различаются.

## Орган, принявший акт

Часто автор документа указывается после слова «Принят», но далеко не всегда, например в указе автор - «Президент Российской Федерации».

В рамках задания необходимо для каждого документа определить орган, его принявший. Наименование органов необходимо извлекать в «нормализованном виде»:

* Ярославская областная дума
* Федеральная служба по надзору в сфере образования и науки
* Правительство Удмуртской Республики
* Департамент охраны окружающей среды и природопользования Ярославской области

В обучающей и тестовой выборке около 200 уникальных органов. В тестовой выборке есть органы, которых нет в обучающей.

Поле authority должно содержать наименование органа в нормализованном виде. При сравнении предсказанного и настоящего органа не учитывается регистр букв, пробелы и символы перевода строки не различаются.

### Оценка качества

Допускается нечеткое совпадение. Метрика качества – средний коэффициент Жаккара.

Коэффициент Жаккара вычисляется следующим образом:

$$ L(s_{extr}, s_{gold}) = \frac{intersect}{offset_{start} + intersect + offset_{end}} $$

где $intersect$ - длина самой длинной общей подстроки у выделенной строки и строки из «золотой метки», $offset_{start}$ - наибольшее количество символов до начала общей подстроки в выделенной строке и в строке из «золотой метки», $offset_{end}$ - наибольшее количество символов от конца общей подстроки до конца строки в выделенной строке и в строке из «золотой метки».

Эта метрика принимает значения от 0 до 1 и штрафует как за выделение более короткой строки чем строка из «золотой метки», так и за выделение более длинной строки.

In [349]:
def ukaz_authority(doc):
    variants = []
    match = re.search(r'президента\s+российской\s+федерации', doc)
    if match:
        variants.append(('президент российской федерации', match.span()[0]))
    
    match = re.search(r'губернатора? ', doc)
    if match:
        index = match.span()[0]
        doc1 = doc[index:]
        authority = doc1.split('\n')[0]
        authority = 'губернатор ' + ' '.join(authority.split(' ')[1:])
        variants.append((authority, index))
    index = doc.find('главы')
    if index != -1:
        doc1 = doc[index:]
        authority = doc1.split('\n')[0]
        authority = 'глава ' + ' '.join(authority.split(' ')[1:])
        variants.append((authority, index))
    match = re.search(r'глава\s*\n', doc)
    if match:
        index = match.span()[0]
        doc1 = doc[index:]
        authority = 'глава ' + doc1.split('\n')[1]
        variants.append((authority, index))
    for author in ['глава ', 'губернатор\n']:
        index = doc.find(author)
        if index != -1:
            doc1 = doc[index:]
            authority = author + doc1.split('\n')[1]
            variants.append((authority, index))
    if variants:
        authority = variants[0][0]
        index = variants[0][1]
        for a, i in variants:
            if i < index:
                authority = a
                index = i 
        return authority
    return ""

In [404]:
PRIKAZ_EXPR = re.compile(r'\n([^\n]*(управление|министерство|департамент|федеральная|служба|комитет|агентство)([^\n]+\n)*)\n')

def prikaz_authority(doc):
    match = PRIKAZ_EXPR.search(doc)
    if match:
        return match.group(1)
    return ""

ZAKON_EXPR = re.compile(r'принят[ \n]([^\n]*(государственн|совет|законодательн|собрание|областн|дум|народн|курултай|президент)[^\n\d]*)')

lemmatizer = {
    'законодательным': 'законодательное', 'областной': 'областная', 'думой': 'дума',
    'народным': 'народное', 'президентом': 'президент', 'государственным': 'государственный', 
    'областное': 'областным', 'законодательной': 'законодательная',
    'собранием': 'собрание', 'советом': 'совет', #'': '', '': '', '': '', '': '', '': '', '': '', '': '',
#     '': '', '': '', '': '', '': '', '': '', '': '', '': '', '': '', '': '',
#     '': '', '': '', '': '', '': '', '': '', '': '', '': '', '': '', '': '',
#     '': '', '': '', '': '', '': '', '': '', '': '', '': '', '': '', '': ''
}

def zakon_authority(doc):
    match = ZAKON_EXPR.search(doc)
    if not match:
        return ""
    tokens = re.sub(r'[^а-я \-]', '', match.group(1)).split(' ')
    authority = ""
    for token in tokens:
        if token in lemmatizer:
            authority += lemmatizer[token] + ' '
        else:
            authority += token + ' '
    authority = re.sub(r'ой( [а-я\-]* ?дума)', r'ая\1', authority)
    return authority

In [405]:
import re

organizations = ['правительство', 'министерство',  'администрация', 'управление', 
                 'комитет', 'департамент', 'губернатора', 'президента']

def extract_authority(doc, doc_type):
    if doc_type == 'федеральный закон':
        return 'Государственная Дума Федерального собрания Российской Федерации'
    doc = '\n' + doc.lower()
    if doc_type == 'указ':
        authority = ukaz_authority(doc)
        if authority:
            return authority
    elif doc_type == 'приказ':
        authority = prikaz_authority(doc)
        if authority:
            return authority
    elif doc_type == 'закон':
        authority = zakon_authority(doc)
        if authority:
            return authority
                
    authority = ""
    for organization in organizations:
        index = doc.find(organization)
        if index != -1:
            doc = doc[index:]
            authority = doc.split('\n')[0]
            if authority.startswith('президента'):
                authority = re.sub('президента', 'президент', authority)
            return authority
    index = doc.find('принят ')
    if index != -1:
        doc = doc[index + 6:]
        authority = doc.split('\n')[0]
        authority = re.sub(r'\d.*', '', authority)

    return authority

In [406]:
PREPR_EXPR = re.compile(r'[^\wа-яА-Я\"\s\.,;:№\-\)\(]')

def preprocess_doc(doc):
    doc = '\n' + doc.lower()
    return PREPR_EXPR.sub('', doc)

In [407]:
NAME_EXPR_1 = re.compile(r'\n((о|об) ([^\n]+\n)+)\n')
NAME_EXPR_2 = re.compile(r'(\"(о|об) [^\"\']+\")')
NAME_EXPR_3 = re.compile(r'(о внесении ([^\n]+\n)+)\n')
LAW_NAME = re.compile(r'о законе [^\"]+(\"([^\n]+\n)+)\n')
NAME_EXPR_4 = re.compile(r'((о|об) ([^\n]+\n)+)\n')

def extract_name(doc, doc_type):
    doc = preprocess_doc(doc)
    names, positions = [], []
    if doc_type == 'закон':
        name = LAW_NAME.search(doc)
        if name:
            names.append(name.group(1))
            positions.append(name.span()[0])
    name = NAME_EXPR_3.search(doc)
    if name:
        names.append(name.group(1))
        positions.append(name.span()[0])
    name = NAME_EXPR_1.search(doc)
    if name:
        names.append(name.group(1))
        positions.append(name.span()[0])
    name = NAME_EXPR_2.search(doc)
    if name:
        names.append(name.group(1))
        positions.append(name.span()[0])
    if not names:
        name = NAME_EXPR_4.search(doc)
        if not name:
            return ""
        return name.group(1)
    name, pos = names[0], positions[0]
    for n, p in zip(names, positions):
        if p < pos:
            pos = p
            name = n
        
    return name

In [408]:
def predict(test, doc_types):
    results = []
    for doc, doc_type in zip(test, doc_types):
        authority = extract_authority(doc, doc_type)
        name = extract_name(doc, doc_type)
        prediction = {"type": "",
                      "date": "",
                      "number": "",
                      "authority": authority,
                      "name": name}
        results.append(prediction)
    return results

In [409]:
from eval_module import quality
import json
import numpy as np

train = []
labeled = []
names = []
with open("train/gold_labels.txt", "r") as read_file:
    for doc_info in read_file.readlines():
        doc_dict = json.loads(doc_info)
        docname = 'train/txts/' + doc_dict['id'] + '.txt'
        names.append(docname)
        with open(docname, 'r') as f:
            train.append(f.read())
        labeled.append(doc_dict['label'])

true_types = np.array([x['type'] for x in labeled])
predicted = predict(train, true_types)
quality(predicted, labeled)

{'date_accuracy': 0.0,
 'number_accuracy': 0.0,
 'type_f1_score': 0.0,
 'name_jaccard': 0.9684145325,
 'authority_jaccard': 0.8717595068,
 'subtasks_improves': 2}

In [410]:
predicted_labels = np.array(predicted)
test_labels = np.array(labeled)
names = np.array(names)
for doc_type in ['федеральный закон', 'постановление', 'приказ', 'распоряжение','закон', 'указ']:
    ids = np.where(true_types == doc_type)
    print(f"{doc_type}: {quality(predicted_labels[ids], test_labels[ids])['authority_jaccard']}")

федеральный закон: 1.0
постановление: 0.8554046136
приказ: 0.6769618532
распоряжение: 0.8566712148
закон: 0.6804457184
указ: 0.9906730137


In [411]:
from eval_module import preprocess, string_jaccard_metric

doc_type = 'закон'
ids = np.where(true_types == doc_type)

y_test = [(x['authority'], x['type']) for x in test_labels[ids]]
y_pred = [x['authority'] for x in predicted_labels[ids]]

wrong_docs = []
i = 0
for test, pred in zip(y_test, y_pred):
    test_, pred_ = preprocess(test[0]), preprocess(pred)
    if string_jaccard_metric([test_], [pred_]) < 0.6:
        wrong_docs.append((names[ids][i], test, pred))
    i += 1
    
for doc_name, test, pred in wrong_docs:
    with open(doc_name, 'r') as f:
        print(f"right value = {test}")
        print(f"wrong value = {pred}")
        print(f"docname = {doc_name}")
        print(f.read())
        print("=====")

right value = ('Государственный Совет Республики Крым', 'закон')
wrong value = государственный совет 
docname = train/txts/c041abe7b625e0eadb9d385cec4801f5aa04f3df.txt
ЗАКОН РЕСПУБЛИКИ КРЫМ

О внесении изменений в некоторые законы Республики Крым
и признании утратившими силу отдельных положений
законодательных актов Республики Крым

Принят
Государственным Советом
Республики Крым 22 октября 2015 года

Статья }

Внести в статью 9 Закона Республики Крым от 29 мая 2014 года
№ 7-3ЗРК "О государственной гражданской службе Республики Крым"
(Ведомости Государственного Совета Республики Крым, 2014, № 1, ст. 74,
ст. 90, № 2, ст. 98, ст. 109, № 3, ст. 218, № 5, ст. 442, № 6, ст. 607; 2015, № 6,
ст. 325; газета "Крымские известия" от 23 сентября 2015 года № 180,
от 6 октября 2015 года № 187) следующие изменения:

в пункте 1 части 5 слово "трех" заменить словом "четырех", слово
"четырех" заменить словом "пяти";

часть 5-1 признать утратившей силу.

Статья 2

Внести в Реестр должностей государственн

* указ - президент, глава, губернатор

* федеральный закон - Государственная Дума Федерального собрания Российской Федерации

* приказ - управление, министерство, департамент, федеральная служба, комитет, агентство

* распоряжение - правительство, администрация, губернатор, президент, глава, дума

* закон - Государственный Совет, Законодательное собрание, областная дума, Законодательная Дума, Государственный Совет, Народное Собрание, Эл Курултай, Президент, областное Собрание

* постановление - Правительство Российской Федерации, Конституционный Суд, Кабинет министров, 
    Правительство, Администрация, Губернатор, Глава, областной совет, областное Собрание,
    Законодательное собрание